# BASIC_CHART

## Overview
This function generates a simple matplotlib chart (e.g., a line or bar chart) from provided numeric data and returns the chart as a PNG image encoded as a base64 string suitable for embedding in Excel or web apps.

## Usage
To use the `BASIC_CHART` function in Excel, enter it as a formula in a cell, specifying your data and optional chart parameters. The function will return a base64-encoded PNG image of the chart, which can be displayed in Excel or other applications that support image embedding.  To insert the image in a viewable manner in Excel you need to use the function dialog and select `Insert result, not formula` option. Using the custom function below will only return the base64 string.

```excel
=BASIC_CHART(data, [chart_type], [title], [xlabel], [ylabel])
```

## Arguments
| Argument    | Type     | Required | Description                                      | Example                |
|-------------|----------|----------|--------------------------------------------------|------------------------|
| data        | 2D list  | Yes      | Numeric data for the chart (rows/columns).       | [[1,2],[2,4],[3,8]]    |
| chart_type  | string   | No       | 'line' or 'bar'. Default is 'line'.              | "line"                 |
| title       | string   | No       | Chart title.                                     | "Growth Curve"         |
| xlabel      | string   | No       | X-axis label.                                    | "X Axis"               |
| ylabel      | string   | No       | Y-axis label.                                    | "Y Axis"               |

## Returns
| Return Value | Type   | Description                                  | Example                        |
|--------------|--------|----------------------------------------------|---------------------------------|
| Chart Image  | string | PNG image as a base64 string for embedding.  | "data:image/png;base64,..."     |
| Error        | string | Error message if calculation fails            | "Error: Data must be numeric."  |

## Limitations
- Only supports 2D numeric data.
- Only 'line' and 'bar' charts are supported.
- Large datasets may result in slow rendering or large images.
- Requires matplotlib and base64 support in the Python environment.

## Benefits
- Quickly visualize data from Excel using Python's matplotlib.
- Embeds charts directly in Excel or web dashboards.
- Customizable chart type and labels.

## Examples

### Simple Line Chart
**Sample Input Data:**
| X | Y |
|---|---|
| 1 | 2 |
| 2 | 4 |
| 3 | 8 |
| 4 | 16|

**Sample Call:**
```excel
=BASIC_CHART(A1:B4, "line", "Growth Curve", "X Axis", "Y Axis")
```
**Sample Output:**
A string starting with `data:image/png;base64,` followed by the base64-encoded PNG image.

### Bar Chart
**Sample Input Data:**
| Category | Value |
|----------|-------|
| A        | 10    |
| B        | 20    |
| C        | 15    |

**Sample Call:**
```excel
=BASIC_CHART(A1:B3, "bar", "Category Values", "Category", "Value")
```
**Sample Output:**
A string starting with `data:image/png;base64,` followed by the base64-encoded PNG image.

In [ ]:
options = {"insert_only":True}

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import io
import base64

def basic_chart(data, chart_type='line', title=None, xlabel=None, ylabel=None):
    """
    Generates a matplotlib chart from 2D numeric data and returns a PNG image as a base64 string.
    Args:
        data (list): 2D list of numbers (Excel range or similar)
        chart_type (str): 'line' or 'bar' (default: 'line')
        title (str): Chart title (optional)
        xlabel (str): X-axis label (optional)
        ylabel (str): Y-axis label (optional)
    Returns:
        str: PNG image as base64 string starting with 'data:image/png;base64,' or error message string
    """
    if not isinstance(data, list) or not data or not isinstance(data[0], list):
        return "Error: Input data must be a 2D list."
    try:
        arr = np.array(data, dtype=float)
    except Exception:
        return "Error: Data must be numeric."
    if arr.ndim != 2 or arr.shape[1] < 2:
        return "Error: Data must have at least two columns (X and Y)."
    x = arr[:, 0]
    y = arr[:, 1]
    plt.figure(figsize=(6, 4))
    if chart_type == 'bar':
        plt.bar(x, y)
    else:
        plt.plot(x, y, marker='o')
    if title:
        plt.title(title)
    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    img_bytes = buf.read()
    img_b64 = base64.b64encode(img_bytes).decode('utf-8')
    return f"data:image/png;base64,{img_b64}"

In [5]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_line_chart():
    data = [
        [1, 2],
        [2, 4],
        [3, 8],
        [4, 16]
    ]
    result = basic_chart(data, chart_type='line', title='Growth Curve', xlabel='X Axis', ylabel='Y Axis')
    assert isinstance(result, str)
    assert result.startswith('data:image/png;base64,')
    assert len(result) > 30

def test_bar_chart():
    data = [
        [1, 10],
        [2, 20],
        [3, 15]
    ]
    result = basic_chart(data, chart_type='bar', title='Category Values', xlabel='Category', ylabel='Value')
    assert isinstance(result, str)
    assert result.startswith('data:image/png;base64,')
    assert len(result) > 30

def test_invalid_data_not_2d():
    data = [1, 2, 3, 4]
    result = basic_chart(data)
    assert isinstance(result, str) and result.startswith('Error:')

def test_invalid_data_non_numeric():
    data = [["a", "b"], ["c", "d"]]
    result = basic_chart(data)
    assert isinstance(result, str) and result.startswith('Error:')

def test_invalid_data_too_few_columns():
    data = [[1], [2], [3]]
    result = basic_chart(data)
    assert isinstance(result, str) and result.startswith('Error:')

ipytest.run()

.........                                                                                        [100%]
5 passed in 0.55s
                                                                                        [100%]
5 passed in 0.55s


<ExitCode.OK: 0>

In [6]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [[1, 2], [2, 4], [3, 8], [4, 16]],
        'line',
        'Growth Curve',
        'X Axis',
        'Y Axis'
    ],
    [
        [[1, 10], [2, 20], [3, 15]],
        'bar',
        'Category Values',
        'Category',
        'Value'
    ]
]

def render_basic_chart_html(data, chart_type, title, xlabel, ylabel):
    result = basic_chart(data, chart_type, title, xlabel, ylabel)
    if isinstance(result, str) and result.startswith("data:image/png;base64,"):
        # Return an HTML <img> tag with the data URL
        return f'<img src="{result}" alt="Chart Image" style="max-width:100%;height:auto;" />'
    return f'<div style="color:red;">{result}</div>'

demo = gr.Interface(
    fn=render_basic_chart_html,
    inputs=[
        gr.Dataframe(headers=["X", "Y"], label="Data", row_count=4, col_count=2, type="array",
                     value=[[1, 2], [2, 4], [3, 8], [4, 16]]),
        gr.Textbox(label="Chart Type ('line' or 'bar')", value='line'),
        gr.Textbox(label="Title", value='Growth Curve'),
        gr.Textbox(label="X Label", value='X Axis'),
        gr.Textbox(label="Y Label", value='Y Axis'),
    ],
    outputs=gr.HTML(label="Chart Image"),
    examples=examples,
    description="Generate a simple matplotlib chart (line or bar) from numeric data and return a PNG image as an HTML <img> tag.",
    flagging_mode="never"
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
